In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
!pip install python_speech_features

  Created wheel for python-speech-features: filename=python_speech_features-0.6-cp36-none-any.whl size=5887 sha256=da79e0eab31f5836e78a08604dfb30092059b3114a1badee14f4c672dfd478ca
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from __future__ import print_function
import sys
import os
import numpy as np


Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
from python_speech_features import mfcc
from sklearn import preprocessing
import scipy.io.wavfile as wav




In [ ]:
def tf_x_init(fan_in, fan_out, dtype=np.float32):
    k =  np.sqrt(6.0 / (fan_in + fan_out))
    return tf.random_uniform((fan_in, fan_out), minval=-k, maxval=k, dtype=dtype)


def sample_bernoulli(probs):
    return tf.nn.relu(tf.sign(probs - tf.random_uniform(tf.shape(probs))))



In [ ]:

class RBM:
    def __init__(self,
                 n_visible,
                 n_hidden,
                 learning_rate=0.01,
                 momentum=0.95,
                 err_function='mse',
                 use_tqdm=False,
                 # DEPRECATED:
                 tqdm=None):
        if not 0.0 <= momentum <= 1.0:
            raise ValueError('momentum should be in range [0, 1]')

       

        self._use_tqdm = use_tqdm
        self._tqdm = None

        if use_tqdm or tqdm is not None:
            from tqdm import tqdm
            self._tqdm = tqdm

        self.n_visible = n_visible
        self.n_hidden = n_hidden
        self.learning_rate = learning_rate
        self.momentum = momentum

        self.x = tf.placeholder(tf.float32, [None, self.n_visible])
        self.y = tf.placeholder(tf.float32, [None, self.n_hidden])

        self.w = tf.Variable(tf_x_init(self.n_visible, self.n_hidden), dtype=tf.float32)
        self.visible_bias = tf.Variable(tf.zeros([self.n_visible]), dtype=tf.float32)
        self.hidden_bias = tf.Variable(tf.zeros([self.n_hidden]), dtype=tf.float32)

        self.delta_w = tf.Variable(tf.zeros([self.n_visible, self.n_hidden]), dtype=tf.float32)
        self.delta_visible_bias = tf.Variable(tf.zeros([self.n_visible]), dtype=tf.float32)
        self.delta_hidden_bias = tf.Variable(tf.zeros([self.n_hidden]), dtype=tf.float32)

        self.update_weights = None
        self.update_deltas = None
        self.compute_hidden = None
        self.compute_visible = None
        self.compute_visible_from_hidden = None

        self._initialize_vars()

        assert self.update_weights is not None
        assert self.update_deltas is not None
        assert self.compute_hidden is not None
        assert self.compute_visible is not None
        assert self.compute_visible_from_hidden is not None

        
        self.compute_err = tf.reduce_mean(tf.square(self.x - self.compute_visible))

        init = tf.global_variables_initializer()
        self.sess = tf.Session()
        self.sess.run(init)

    def _initialize_vars(self):
        pass

    def get_err(self, batch_x):
        return self.sess.run(self.compute_err, feed_dict={self.x: batch_x})



    def partial_fit(self, batch_x):
        self.sess.run(self.update_weights + self.update_deltas, feed_dict={self.x: batch_x})

    def fit(self,
            data_x,
            n_epoches=10,
            batch_size=10,
            shuffle=True,
            verbose=True):
        assert n_epoches > 0

        n_data = data_x.shape[0]

        if batch_size > 0:
            n_batches = n_data // batch_size + (0 if n_data % batch_size == 0 else 1)
        else:
            n_batches = 1

        if shuffle:
            data_x_cpy = data_x.copy()
            inds = np.arange(n_data)
        else:
            data_x_cpy = data_x

        errs = []

        for e in range(n_epoches):
            if verbose and not self._use_tqdm:
                print('Epoch: {:d}'.format(e))

            epoch_errs = np.zeros((n_batches,))
            epoch_errs_ptr = 0

            if shuffle:
                np.random.shuffle(inds)
                data_x_cpy = data_x_cpy[inds]

            r_batches = range(n_batches)

            if verbose and self._use_tqdm:
                r_batches = self._tqdm(r_batches, desc='Epoch: {:d}'.format(e), ascii=True, file=sys.stdout)

            for b in r_batches:
                batch_x = data_x_cpy[b * batch_size:(b + 1) * batch_size]
                self.partial_fit(batch_x)
                batch_err = self.get_err(batch_x)
                epoch_errs[epoch_errs_ptr] = batch_err
                epoch_errs_ptr += 1

            if verbose:
                err_mean = epoch_errs.mean()
                if self._use_tqdm:
                    self._tqdm.write('Train error: {:.4f}'.format(err_mean))
                    self._tqdm.write('')
                else:
                    print('Train error: {:.4f}'.format(err_mean))
                    print('')
                sys.stdout.flush()

            errs = np.hstack([errs, epoch_errs])

        return errs

    def get_weights(self):
        return self.sess.run(self.w),\
            self.sess.run(self.visible_bias),\
            self.sess.run(self.hidden_bias)

    def save_weights(self, filename, name):
        saver = tf.train.Saver({name + '_w': self.w,
                                name + '_v': self.visible_bias,
                                name + '_h': self.hidden_bias})
        return saver.save(self.sess, filename)

    def set_weights(self, w, visible_bias, hidden_bias):
        self.sess.run(self.w.assign(w))
        self.sess.run(self.visible_bias.assign(visible_bias))
        self.sess.run(self.hidden_bias.assign(hidden_bias))

    def load_weights(self, filename, name):
        saver = tf.train.Saver({name + '_w': self.w,
                                name + '_v': self.visible_bias,
                                name + '_h': self.hidden_bias})
        saver.restore(self.sess, filename)


In [ ]:

class GBRBM(RBM):
    def __init__(self, n_visible, n_hidden, **kwargs):
       

        RBM.__init__(self, n_visible, n_hidden, **kwargs)

    def _initialize_vars(self):
        hidden_p = tf.nn.sigmoid(tf.matmul(self.x, self.w) + self.hidden_bias)
        visible_recon_p = tf.matmul(sample_bernoulli(hidden_p), tf.transpose(self.w)) + self.visible_bias

        

        hidden_recon_p = tf.nn.sigmoid(tf.matmul(visible_recon_p, self.w) + self.hidden_bias)

        positive_grad = tf.matmul(tf.transpose(self.x), hidden_p)
        negative_grad = tf.matmul(tf.transpose(visible_recon_p), hidden_recon_p)

        def f(x_old, x_new):
            return self.momentum * x_old +\
                   self.learning_rate * x_new * (1 - self.momentum) / tf.to_float(tf.shape(x_new)[0])

        delta_w_new = f(self.delta_w, positive_grad - negative_grad)
        delta_visible_bias_new = f(self.delta_visible_bias, tf.reduce_mean(self.x - visible_recon_p, 0))
        delta_hidden_bias_new = f(self.delta_hidden_bias, tf.reduce_mean(hidden_p - hidden_recon_p, 0))

        update_delta_w = self.delta_w.assign(delta_w_new)
        update_delta_visible_bias = self.delta_visible_bias.assign(delta_visible_bias_new)
        update_delta_hidden_bias = self.delta_hidden_bias.assign(delta_hidden_bias_new)

        update_w = self.w.assign(self.w + delta_w_new)
        update_visible_bias = self.visible_bias.assign(self.visible_bias + delta_visible_bias_new)
        update_hidden_bias = self.hidden_bias.assign(self.hidden_bias + delta_hidden_bias_new)

        self.update_deltas = [update_delta_w, update_delta_visible_bias, update_delta_hidden_bias]
        self.update_weights = [update_w, update_visible_bias, update_hidden_bias]

        self.compute_hidden = tf.nn.sigmoid(tf.matmul(self.x, self.w) + self.hidden_bias)
        self.compute_visible = tf.matmul(self.compute_hidden, tf.transpose(self.w)) + self.visible_bias
        self.compute_visible_from_hidden = tf.matmul(self.y, tf.transpose(self.w)) + self.visible_bias


In [ ]:
# data preprocessor
# Here Bulk data is preprocessed to use in training


In [ ]:
shift = 1
merge_limit=4
dataset = []


#name of the file where urbm model learned weights and biases will be saved.
save_weights_path = "/content/gdrive/My Drive/SpeakerVerification/SavedModels/urbm_model" 


lr  = 0.001
input_dim =80
hidden_dim = 400
batch_size = 100
nb_epoch = 100

# this is the directory where all test wav data
# each file corresponds to each speaker
path_dir = "/content/gdrive/My Drive/SpeakerVerification/Data/" 
file_list=os.listdir(path_dir)

print(file_list)



['file_example_WAV_2MG.wav', 'taunt.wav', 'StarWars60.wav', 'StarWars3.wav', 'ortalking.wav', '3peopletalking.wav', 'OSR_us_000_0010_8k.wav', 'OSR_us_000_0011_8k.wav', 'OSR_us_000_0012_8k.wav', 'OSR_us_000_0013_8k.wav', 'OSR_us_000_0030_8k.wav', 'OSR_us_000_0031_8k.wav', 'OSR_us_000_0032_8k.wav', 'OSR_us_000_0034_8k.wav', 'OSR_us_000_0035_8k.wav', 'OSR_us_000_0036_8k.wav', '2.wav', '3.wav']


In [ ]:
# vector 
import numpy as np

supervector_list=[]

for f in file_list:
    if(f.endswith(".wav")==False):
        continue

    gbrbm = GBRBM(n_visible=input_dim, n_hidden=hidden_dim, learning_rate=lr, momentum=0.95, use_tqdm=True)
    gbrbm.load_weights(save_weights_path+f,'weights'+f)
    w,b_bias,h_bias = gbrbm.get_weights()

    w=np.reshape(w,80*400)
    supervector=np.concatenate((w,b_bias,h_bias),axis=None)
    # print(supervector.size)
    # supervector=preprocessing.scale(supervector) # mean normalization of supervectors
    # print(supervector)
    supervector_list.append(supervector)


# fit current size of the ram
trunk_sv_list=[]
for item in supervector_list:
    new_item=item[:15000]
    trunk_sv_list.append(new_item)
trunk_vectors= np.array(trunk_sv_list)
print(trunk_vectors.shape)
#

supervector_data=np.array(supervector_list)
#normal
supervector_data=preprocessing.scale(supervector_data, axis=0)

# gbrbm = GBRBM(n_visible=input_dim, n_hidden=hidden_dim, learning_rate=lr, momentum=0.95, use_tqdm=True)
# gbrbm.load_weights(save_weights_path,'weights')
# w,b_bias,h_bias = gbrbm.get_weights()

# w=np.reshape(w,80*400)
# urbm_supervector=np.concatenate((w,b_bias,h_bias),axis=None)
# print(supervector.size)
# u_vector=preprocessing.scale(urbm_supervector) # mean normalization of supervectors


Instructions for updating:
Use `tf.cast` instead.
INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/SpeakerVerification/SavedModels/urbm_modelfile_example_WAV_2MG.wav
INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/SpeakerVerification/SavedModels/urbm_modeltaunt.wav
INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/SpeakerVerification/SavedModels/urbm_modelStarWars60.wav
INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/SpeakerVerification/SavedModels/urbm_modelStarWars3.wav
INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/SpeakerVerification/SavedModels/urbm_modelortalking.wav
INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/SpeakerVerification/SavedModels/urbm_model3peopletalking.wav
INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/SpeakerVerification/SavedModels/urbm_modelOSR_us_000_0010_8k.wav
INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_data.py:173: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_data.py:190: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


In [ ]:
# PCA whitening and dimensionality reduction  ...
import numpy as np
from sklearn.decomposition import PCA
pca = PCA()
X=supervector_data
pca.fit(X)
rbm_vectors= pca.transform(X)

print(rbm_vectors.shape)
print(rbm_vectors)

(18, 18)
[[-7.77966833e+00  1.34871964e+02 -9.05001700e-01  1.85079132e+02
  -7.12254944e+01 -3.97905884e+01 -7.54837179e+00  6.68858814e+00
   1.14842623e-01  9.61022496e-01 -1.43848801e+00  1.59038270e+00
   1.39129257e+00  1.88803887e+00  1.18296874e+00 -3.48188102e-01
  -1.30440235e+00 -2.25650147e-05]
 [-8.55479336e+00  4.34726105e+01  5.48930073e+00  1.53829756e+01
   2.39334698e+01  8.35223160e+01  2.92894955e+01 -4.74360695e+01
   4.49906206e+00  1.74747410e+01 -8.30813599e+00  8.14800739e+00
  -1.36763036e+00  3.30426331e+01  2.43540039e+01 -2.23404522e+01
  -2.18531132e+01  7.75735825e-05]
 [-1.44103813e+01  1.17376587e+02  1.92473557e+02 -1.18840149e+02
  -5.08542633e+01 -2.42987137e+01 -4.66350365e+00  5.56161356e+00
  -2.38673973e+00  8.49018693e-01  2.00056791e+00  1.63096297e+00
   3.46642327e+00  2.24365091e+00  1.79245436e+00 -3.65388244e-02
  -3.20069313e+00  6.12121075e-05]
 [-6.32685041e+00  4.86093521e+01  1.83094845e+01  1.17979765e+01
   1.11907864e+01  7.5772621

In [ ]:
# clustering tasks starts here ...
from sklearn.cluster import AgglomerativeClustering
X=rbm_vectors
clustering = AgglomerativeClustering(n_clusters=6).fit(X)


print(clustering.labels_)

[5 0 4 0 0 3 1 1 1 1 2 2 2 2 2 2 0 0]
